# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [1]:
# off-the-shelf 방식
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

# 변수:값 -> (prompt_template) prompt -> (model) 응답결과 -> (parser) -> 최종결과

In [2]:
from langchain import LLMChain
chain = LLMChain(
    prompt=prompt_template,
    llm=model,
    output_parser=parser
)
response = chain.invoke({"item":"가방", "count":5})

/var/folders/69/bt3_5fns57n_cp5_ydbrhfh80000gn/T/ipykernel_67577/1602056365.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [86]:
response

{'item': '가방',
 'count': 5,
 'text': '물론입니다! 가방에 어울리는 이름 5개를 제안해 드릴게요.\n\n1. **여행자 (Traveler)** - 여행을 함께할 수 있는 가방에 어울리는 이름.\n2. **스타일리스트 (Stylist)** - 패션에 포인트를 줄 수 있는 세련된 가방에 적합한 이름.\n3. **하루 (Haru)** - 일상적인 사용을 고려한 가볍고 실용적인 가방에 어울리는 이름.\n4. **비비안 (Vivian)** - 우아함과 실용성을 겸비한 여성 가방에 적합한 이름.\n5. **모험가 (Adventurer)** - 다양한 환경에서 사용할 수 있는 튼튼한 가방을 위한 이름.\n\n이 이름들이 가방의 특성을 잘 표현하기를 바랍니다!'}

# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassThrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

#### Runnable 예제

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_core.runnables import Runnable    # All Runnable의 TOP
# 사용자 정의 Runnable
class MyRunnable(Runnable):
    
    def invoke(self, input_data : str, config : dict = None):
        # invoke() : 구현하는 Runnable이 해야하는 작업을 구현하는 method
        # input_data : 입력값
        # config : 일할 때 필요한 설정 값

        if config is not None and config.get("lang") == "en":
            return f"Explain {input_data} in one sentences."
        return f"{input_data}에 대해서 한 문장으로 설명 부탁~ 해요 ~"

In [9]:
my_runnable = MyRunnable()
my_runnable.invoke("사과")
my_runnable.invoke("콤퓨타")
my_runnable.invoke("Apple", {"lang" : "en"})

'Explain Apple in one sentences.'

In [10]:
from langchain_openai import ChatOpenAI

my_runnable = MyRunnable()
model = ChatOpenAI(model_name = "gpt-4o-mini")

prompt = my_runnable.invoke("apple", {"lang" : "en"})
response = model.invoke(prompt)

print(response)

content='An apple is a round, typically red, green, or yellow fruit produced by the apple tree (Malus domestica), known for its sweet to tangy flavor and crisp texture, often eaten raw or used in various culinary dishes.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 13, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgkAaQSYljlbEV6yQIaxDgkvTByF6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--806b5342-929d-4665-acc3-fb38a082c490-0' usage_metadata={'input_tokens': 13, 'output_tokens': 47, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [11]:
prompt = my_runnable.invoke("langchain")
response = model.invoke(prompt)

print(response)

content='LangChain은 자연어 처리 모델을 사용하여 다양한 애플리케이션을 구축하고, 언어 모델과 외부 데이터 소스를 통합하는 데 도움을 주는 프레임워크입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 21, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkAbOO6vAh3QdQTWzgkEqWobMAGr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--4593b76b-354e-45aa-9aaa-e05e4631786b-0' usage_metadata={'input_tokens': 21, 'output_tokens': 43, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [12]:
# chain -> Runnable | Runnable | Runnable
chain = my_runnable | model

# chain 호출
res = chain.invoke("과일 배")
print(res)

content='배는 상큼하고 달콤한 맛을 지닌 과일로, 주로 여름철에 수확되어 생식으로 즐기거나 다양한 요리에 사용됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 22, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkAcHZZIDLhJ4cA0nbnftPNwHIb4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a32f9490-333c-4ad4-893f-bb5bc8f967c8-0' usage_metadata={'input_tokens': 22, 'output_tokens': 38, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [13]:
print(res.content)

배는 상큼하고 달콤한 맛을 지닌 과일로, 주로 여름철에 수확되어 생식으로 즐기거나 다양한 요리에 사용됩니다.


In [93]:
# 기본 체인 구성 : prompt_template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from dotenv import load_dotenv
load_dotenv()

# role: system, user/human, ai/assistant
# system: 채팅 전체에 적용되는 공통 지침을 지정하는 role
prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 오랜 경력의 한국관광가이드입니다. 여행객들에게 설명하듯이 친절하게 답변을 해주세요."),
        ("human", "{query}")
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)
guide_chain = prompt_template | model | StrOutputParser()
print(type(guide_chain))        # RunnableSequence => chain도 다른 chain의 구성요소로 포함될 수 있다.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [94]:
query = "서울에서 꼭 먹어야되는 돈까스 맛집을 세곳만 알려줘."
response = guide_chain.invoke({"query":query})

In [95]:
print(response)

서울에는 맛있는 돈까스 맛집이 정말 많아요! 여행객 여러분이 드셔보셔야 할 최고의 돈까스 맛집 세 곳을 추천해 드릴게요.

1. **우래옥** - 서울 종로구에 위치한 '우래옥'은 전통적인 스타일의 돈까스를 제공하는 곳입니다. 여기 돈까스는 두툼한 pork loin을 사용해 바삭한 튀김옷과 함께 육즙이 풍부해 먹을 때마다 감동을 느낄 수 있어요. 특히 매콤한 초고추장과 함께 먹으면 한층 더 맛이 살아납니다.

2. **미쓰양평해장국** - 강남구에 위치한 이곳은 돈까스와 해장국을 전문으로 하는 곳이에요. '미쓰양평해장국'의 돈까스는 신선한 재료로 만들어져 바삭하면서도 부드러운 식감을 자랑해요. 특히 해장국과 함께 먹는 조합이 일품이랍니다!

3. **돈까스 제왕** - 중구 회현동에 있는 이곳은 퓨전 스타일의 돈까스를 제공하고 있어요. 특히 여기의 '까뜨리 돈까스'는 갈비맛이 나는 특별한 소스와 함께 제공되는데, 돈까스의 맛에 새로운 변화를 주는 색다른 경험을 선사합니다.

각 맛집마다 개성이 있으니, 꼭 한 번 들러보세요! 맛있는 돈까스와 함께 서울의 맛을 즐기며 행복한 여행이 되시길 바랍니다.


In [ ]:
while 1:
    query = input("질문")
    if query == "!quit":
        resp = guide_chain.invoke({"query":query})
        print("User:", query)
        print("AI:", resp)
        print("-"*50)

#### RunnableLambda 예제

In [4]:
from langchain_core.runnables import RunnableLambda
my_runnable2 = RunnableLambda(
lambda input_data : f"{input_data}을(를) 한 문장으로 설명해줘"
)
my_runnable2.invoke("LLM")

'LLM을(를) 한 문장으로 설명해줘'

In [5]:
chain = my_runnable2 | model
chain.invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 대량의 텍스트 데이터를 기반으로 학습하여 인간처럼 자연어를 이해하고 생성할 수 있는 인공지능 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 20, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkBVh4krSRW804yNEZpV0war3n5r', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b7a51c26-a37e-4a0b-9b43-f11df19c4f06-0', usage_metadata={'input_tokens': 20, 'output_tokens': 38, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
def sum(nums):
    return nums[0] + nums[1]

my_runnable2=RunnableLambda(sum)
my_runnable2.invoke({0:10, 1:20})

# invoke(입력데이터:str|dict, 설정정보:dict)
# 입력데이터가 여러개일 경우 dict등의 자료구조를 이용해서 받는다.

30

#### RunnablePassThrough 예제

In [9]:
# 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")
RunnablePassthrough().invoke({"key":"value"})
# 무슨 타입이던지 받은 값을 그대로 넘김

{'key': 'value'}

In [11]:
# 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 그대로 전달
# 입력으로 dictionary를 받아 거기에 item 추가
# RunnablePassthrough.assign(key=Runnable, key1=Runnable ,......)
# -> 받은 dictionary에 추가해서 다음으로 전달
address_runnable = RunnableLambda(lambda x: "서울시 금천구")
phone_runnable = RunnableLambda(lambda x: "010-1111-2222")

RunnablePassthrough.assign(address=address_runnable, phone=phone_runnable).invoke({"name":"홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

#### RunnableSequence 예제

In [ ]:
from langchain_core.runnables import RunnableSequence
run1 = RunnableLambda(lambda x: x+1)
run2 = RunnableLambda(lambda x: x*2)

chain = run1 | run2
chain.invoke(30)

62

In [13]:
chain2 = RunnableSequence(run1,run2)        # (prompt_template, model, output_parser)
chain2.invoke(100)

202

#### RunnableParallel 예제

In [4]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough

run1 = RunnableLambda(lambda x: x+1)
run2 = RunnableLambda(lambda x: x*2)
run3 = RunnableLambda(lambda x: x//3)

runnable = RunnableParallel(
    {
        "result1":run1,
        "result2":run2,
        "result3":run3,
        "result4":RunnablePassthrough()                         # 전달받은 값 그대로 반환
    }
)
# runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary를 반환
runnable.invoke(20)

{'result1': 21, 'result2': 40, 'result3': 6, 'result4': 20}

In [5]:
run0 = RunnableLambda(lambda x: x+100)
chain = run0 | {
    "result1":run1,
    "result2":run2,
    "result3":run3,
}
chain.invoke(20)

{'result1': 121, 'result2': 240, 'result3': 40}

#### LCEL Chain 예제

In [62]:
# 음식 이름을 받아서 레시피를 "영어"로 출력하는 chain을 구성
# prompt template -> model -> output parser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent
from dotenv import load_dotenv
load_dotenv()

# lang = RunnableLambda(lambda x: "영어")

prompt_template = PromptTemplate(
    template=dedent("""
        # Instrucion
        당신은 숙련된 미슐렝 셰프입니다. 요청한 음식의 레시피를 작성해 주세요.
        # Input data
        음식이름 : {food}
        # Output indicator
        - 레시피는 영어로 작성해 주세요.
""")
)

model = ChatOpenAI(model_name="gpt-4o-mini")
food_chain = prompt_template | model | StrOutputParser()

In [63]:
response = food_chain.invoke({"food":"pasta"})
print(response)

## Recipe for Classic Pasta

### Ingredients:
- 400g (14 oz) of dried pasta (spaghetti, fettuccine, or penne)
- 2 tablespoons of olive oil
- 3 cloves of garlic, minced
- 1 can (400g) of diced tomatoes (or 4 medium ripe tomatoes, chopped)
- 1 teaspoon of sugar (optional, to balance acidity)
- Salt and pepper to taste
- Fresh basil leaves, for garnish
- Grated Parmesan cheese, for serving

### Instructions:

1. **Cook the Pasta:**
   - In a large pot, bring salted water to a boil. 
   - Add the dried pasta and cook according to package instructions until al dente.
   - Reserve 1 cup of pasta cooking water, then drain the pasta and set aside.

2. **Prepare the Sauce:**
   - In a large skillet over medium heat, add the olive oil.
   - Once hot, add the minced garlic and sauté for about 1 minute until fragrant, being careful not to burn it.
   - Add the diced tomatoes (and sugar, if using) to the skillet. Stir to combine and let it simmer for about 10 minutes, allowing the sauce to thicken 

In [34]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv()

prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 오랜 경력의 미슐랭 셰프입니다. 학생들이 원하는 음식의 레시피를 {lang}로 알려주세요."),
        ("human", "{food}")
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)
food_chain = prompt_template | model | StrOutputParser()

In [37]:
response = food_chain.invoke({"lang":"히브리어", "food":"pasta"})
print(response)

כמובן! הנה מתכון פשוט להכנת פסטה בבית:

### מצרכים:
- 2 כוסות קמח (עדיף קמח דורום)
- 3 ביצים
- 1/2 כפית מלח
- 1 כף שמן זית (אופציונלי)

### הוראות:
1. **הכנת הבצק:**
   - שימו את הקמח על השולחן או בקערה גדולה, וצרו גומה במרכז.
   - הוסיפו את הביצים, המלח ושמן הזית וערבבו בעזרת מזלג עד שהתערובת מתחילה להתאחד.
   - בעזרת הידיים, לושו את הבצק עד שהוא חלק וגמיש (כ-10 דקות). אם הבצק דביק, הוסיפו עוד מעט קמח.

2. **מנוחה:**
   - עטפו את הבצק בניילון נצמד או בנייר מוקה והניחו לו לנוח במקרר למשך 30 דקות.

3. **הכנת הפסטה:**
   - לאחר מנוחה, חלקו את הבצק ל-4 חלקים.
   - רדו כל חלק לעובי הרצוי (בעזרת מכונת פסטה או רול-עץ).
   - חתכו לעוביים הרצויים (ספגטי, פטוצ'יני וכו').

4. **בישול:**
   - בשלו סיר עם מים מומלחים. כאשר המים רותחים, הוסיפו את הפסטה ובשלו כ-2-3 דקות (או עד שהיא צפה).
   - סננו את הפסטה והגישו עם רוטב אהוב עליכם.

בתיאבון!


In [75]:
# 번역 chain -> 입력된 내용을 지정한(입력한) 언어로 번역하는 체인
# prompt template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv()

prompt_template2 = ChatPromptTemplate(
    messages=[
        ("system", "당신은 다개국어 능력자입니다. {content}을(를) {language}로 알려주세요.")
    ]
)
model = ChatOpenAI(model_name="gpt-4o-mini")
translate_chain = prompt_template2 | model | StrOutputParser()

In [76]:
# translate_chain.invoke({"content":"안녕하세요", "language":"한국어"})
response = "배고파"
res = translate_chain.invoke({"content":response, "language":"독일어"})
res

'"배고파"는 독일어로 "Ich habe Hunger"라고 합니다.'

## Chain과 Chain간의 연결

In [77]:
# food_chain + translate_chain
# food_chain_prompt : 변수 - food
# translate_chain : 변수 - language, content

# RunnableParallel({"key":Runnable, "key2":Runnable})
# LCEL에서 RunnableParallel => {"key":Runnable, "key2":Runnable} |
chain = {
            "content":food_chain,
            "language":RunnableLambda(lambda x: x['language'])
        } | translate_chain

In [78]:
res = chain.invoke({"food":"파스타", "language":"영어"})
# food -> food_chain, language -> translate_chain
# food_chain 최종결과(레시피) => {"content":레시피} 전달 -> translate_chain

In [79]:
print(res)

Sure! Here is the pasta recipe in English:

### Pasta Recipe

#### Ingredients:
- 400g pasta (spaghetti, fettuccine, or your choice)
- 2 tablespoons olive oil
- 3 cloves garlic, minced
- 1 teaspoon red pepper flakes (optional)
- 1 can (400g) crushed tomatoes
- 1 teaspoon salt (adjust to taste)
- ½ teaspoon black pepper
- 1 tablespoon sugar (to balance acidity)
- Fresh basil leaves, torn (about 10-15 leaves)
- Grated Parmesan cheese, for serving
- Optional: Cooked protein of choice (chicken, shrimp, or chickpeas for a vegetarian option)

#### Instructions:

1. **Cook the Pasta:**  
   Bring a large pot of salted water to a boil. Add the pasta and cook according to package instructions until al dente. Reserve 1 cup of pasta water, then drain the pasta and set aside.

2. **Prepare the Sauce:**  
   In a large skillet, heat the olive oil over medium heat. Add the minced garlic and red pepper flakes (if using) and sauté for about 1 minute until fragrant, ensuring not to burn the garlic.

3.

In [73]:
chain.invoke({"food":"냉면", "language":"몽골어"})

'### Наенгмён (Хүйтэн гоймон) жор\n\n#### Орц:\n\n**Гоймон для:**\n- 200 г наенгмён гоймон (буурцагны гоймон)\n- Ус (буцалгахынため)\n- Мөс (хүйтэн усанд хөргөхийнため)\n\n**Шөл для:**\n- 4 аяга үхрийн шөл (эсвэл 4 аяга ус, 2-3 уут хүчил, соёос нэмж хийж болно)\n- 1 хоолны халбага соёос\n- 1 хоолны халбага хүчил (яншуй эсвэл бургар ханд)\n- 1 цайны халбага элсэн чихэр\n- Давс амтлах\n\n**Гарнирын для:**\n- 1 газрын төлд, нарийн хэрчиж\n- 1 Азийн намар, нимгэн хэрчиж (сонголтоор)\n- Вандуй хатаасалсан (хагас)\n- Тахлын бургас (сонголтоор, исгэлэн амтанд)\n- Сусамын үр (чулуугаар)\n- Солонгосын гичний соус (сонголтоор)\n\n#### Заавар:\n\n1. **Гоймоныг бэлдэх:**\n   - Том тогоо ус буцалгана. Наенгмён гоймоныг хийж, багцын зааврын дагуу, ихэвчлэн 3-5 минутын турш болгож бэлдэнэ.\n   - Бэлдсэн гоймоныг шүүж, хөргөсөн усны доор усанд хөшүүлбэл, боловсруулалтыг зогсоож, илүүдэл крахмалыг арилгана. Мөсөөр хийсэн усанд шилжүүлж хөргөж тавина.\n\n2. **Шөл бэлтгэх:**\n   - Тогоонд үхрийн шөлийг (эсвэ

# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

In [8]:
from langchain_core.runnables import RunnableLambda
# 기존에 정의된 함수를 Runnable로 정의
def plus(num1, num2, num3):
    return num1+num2+num3

# run1.invoke(1,2,3)   -> invoke는 3개 못받음

def wrapper_plus(nums:list):
    return plus(nums[0], nums[1], nums[2])

run1 = RunnableLambda(wrapper_plus)
run1.invoke([1,2,3])
# invoke(input_data:첫번째, config:RunnableConfig-설정정보=None)

6

In [2]:
# 소재를 이용해서 이야기를 생성하는 chain
# 장문을 입력 받아서 요약하는 chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from textwrap import dedent
load_dotenv()

True

In [13]:
story_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)
story_prompt_template = PromptTemplate(
    template=dedent("""
            # Instruction
            당신은 아이들을 위한 이야기를 창작하는 스토리텔러입니다.
            주어진 소재로 잠자리에서 아이들에게 들려줄 재미있는 이야기를 만들어 주세요.
                    
            # Input Data
            소재 : {topic}
            
            # Output Indicator
            - 이야기는 30문장 이내로 구성해 주세요.
            - 이야기를 구어체로 작성해 주세요.
""")
)
story_chain = story_prompt_template | story_model | StrOutputParser()

In [14]:
ssummary_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
# 요약 체인 구성
summary_prompt = PromptTemplate(
    template=dedent("""
            # Instruction
            주어진 문장 내용을 2문장으로 요약해주세요.
            
            # Input Data
            {content}
""")
)
summary_chain = summary_prompt | ssummary_model | StrOutputParser()

In [15]:
chain = story_chain | summary_chain
chain.invoke({"topic":"국밥"})

'국밥 할아버지는 마을에서 가장 맛있는 국밥을 끓이며 아이들에게 특별한 비법을 전수했습니다. 아이들은 할아버지와 함께 국밥을 배우며 꿈을 키우고, 마을의 자랑이 되는 국밥을 만들게 되었습니다.'

In [16]:
from langchain_core.runnables import chain
# 사용자 정의 체인 -> 흐름을 원하는 구조로 정의할 수 있다.
@chain                  # decorator를 통해 runnable에 넣지 않고도 type을 runnable로 설정 가능
def custom_chain(topic:str)->dict[str,str]:
    # story_chain과 summary_chain을 원하는 흐름으로 구현
    # story_chain의 결과와 summary_chain의 결과를 모두 반환하도록 처리
    story = story_chain.invoke({"topic":topic})
    summary = summary_chain.invoke({"content":story})

    return {"전체이야기":story, "이야기요약":summary}

In [7]:
type(custom_chain)

langchain_core.runnables.base.RunnableLambda

In [17]:
res = custom_chain.invoke("물")

In [9]:
print(type(res))
print(res.keys())

<class 'dict'>
dict_keys(['전체이야기', '이야기요약'])


In [18]:
print(res["전체이야기"])

옛날 옛적, 어느 작은 마을에 '물방울'이라는 이름의 소년이 살고 있었어요. 물방울은 아주 특별한 능력이 있었답니다. 그는 물을 다룰 수 있는 힘을 가지고 있었죠! 마을 사람들이 마실 물이 부족하다고 걱정할 때, 물방울이 나타나곤 했어요. 

“걱정하지 마세요!” 물방울은 말했다. “제가 시냇물을 불러올게요!” 그리고 손을 흔들자 시냇물은 그의 곁으로 쏠려왔죠. 사람들이 너무 놀라서 "대단해!"라고 외쳤어요. 

어느 날, 마을에 큰 문제가 생겼어요. 마을 근처의 강물이 마르기 시작한 거예요. 사람들은 비를 기다렸지만, 비는 오지 않았어요. 그러자 물방울은 결심했어요. “내가 강물을 다시 살려야 해!” 

물방울은 마을 꼭대기에 있는 높은 언덕으로 올라갔어요. 거기서 그는 큰 물의 신에게 기도를 했죠. “물의 신님, 도와주세요!” 그러자 물의 신이 나타났어요. 

“물방울아, 넌 용감하고 순수한 마음을 가졌구나. 너의 소원을 들어줄게.” 물의 신이 말했어요. 갑자기 하늘이 흐려지기 시작하더니, 빗방울들이 쏟아지기 시작했어요. 사람들은 모두 밖으로 나와서 비를 맞으며 놀랐어요! 

“우와! 물이 온다!” 사람들이 소리쳤고, 물방울은 신이 주신 비로 강물을 다시 채웠어요. 강물이 흘러넘치자 마을 사람들은 기뻐하며 서로 포옹했어요. 

그날 이후로 물방울은 마을의 영웅이 되었어요. 사람들은 그를 위해 매년 ‘물의 날’을 만들어 축제를 열었답니다. 사람들은 춤추고 노래하며 물의 소중함을 기념했어요. 물방울은 더이상 외롭지 않았고, 모두와 함께 행복하게 살았어요. 

“기억해요, 물은 우리 삶의 원천이에요,” 물방울은 항상 말했답니다. 이제 마을의 사람들도 물을 아끼고 소중히 여겼어요. 그렇게 물방울과 마을은 언제까지나 함께 행복하게 살았답니다. 

자, 이제 잘 자요! 물이 만들어낸 소중한 이야기를 꿈꾸며, 우리도 그 소중함을 기억해봐요!


In [19]:
print(res["이야기요약"])

물방울이라는 소년은 물을 다룰 수 있는 특별한 능력을 가지고 있어 마을의 물 부족 문제를 해결하며 영웅이 되었습니다. 그는 물의 신의 도움으로 비를 내리게 하여 강물을 다시 채우고, 마을 사람들은 매년 '물의 날'을 기념하며 물의 소중함을 잊지 않게 되었습니다.


# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
# SQLite -> Python 내장 DB
# set_llm_cache(InMemoryCache())                                   # memory에 대화 저장할 수 있는 cache 생성
set_llm_cache(SQLiteCache("llm_cache.sqlite"))                     # 파일로 저장
model = ChatOpenAI(model_name="gpt-4o-mini")
response = model.invoke("주요 프로그래밍 언어 5개를 소개해주세요.")         # 질문이 바뀌지 않는다면 저장된 값을 바로 불러오므로 0초 소요 / 질문이 살짝이라도 바뀐다면 다시 10초가량 소요

In [2]:
print(response.content)

다음은 주요 프로그래밍 언어 5가지를 소개합니다:

1. **Python**:
   - **특징**: 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 초보자에게 적합합니다. 데이터 분석, 웹 개발, 인공지능, 자동화 등 다양한 분야에 사용됩니다.
   - **주요 라이브러리**: NumPy, pandas, TensorFlow, Flask, Django 등.

2. **Java**:
   - **특징**: 객체지향 프로그래밍 언어로, 플랫폼 독립성을 갖고 있어 "Write Once, Run Anywhere"라는 슬로건을 가지고 있습니다. 엔터프라이즈 애플리케이션, 모바일 앱(특히 안드로이드), 웹 서버 개발에 많이 쓰입니다.
   - **주요 프레임워크**: Spring, Hibernate 등.

3. **JavaScript**:
   - **특징**: 주로 웹 개발에서 사용되는 스크립트 언어로, 클라이언트 사이드와 서버 사이드 양쪽 모두에서 활용됩니다. 사용자 인터페이스 및 동적인 웹 페이지를 만드는 데 필수적입니다.
   - **주요 라이브러리 및 프레임워크**: React, Angular, Vue.js 등.

4. **C++**:
   - **특징**: C 언어를 기반으로 한 고급 언어로, 객체지향 프로그래밍, 메모리 제어, 시스템 프로그래밍에 강점이 있습니다. 게임 개발, 시스템 소프트웨어, 고성능 애플리케이션 등에서 널리 사용됩니다.
   - **주요 라이브러리**: STL(Standard Template Library), Boost 등.

5. **C#**:
   - **특징**: Microsoft에서 개발한 언어로, 주로 .NET 프레임워크와 함께 사용됩니다. 윈도우 애플리케이션, 게임 개발(Unity 엔진을 사용한) 등에 적합합니다. 객체지향 프로그래밍과 강력한 타입 검사를 제공합니다.
   - **주요 프레임워크**: ASP.NET, Xamarin 등.

이 언어들은 각각의 특성과 용도에 따라 다양한 산업과 프로젝트에서 널리 사용되고 있

In [3]:
# streaming 방식(한 단어씩 보여지는, 응답이 생성되는 것이 보여지는) 응답처리
# - LLM 응답을 기다리지 않고 실시간으로 생성되는 토큰/청크를 받아서 처리한다.
# - model.invoke(input, config)->(return type:)응답데이터 : 응답 완료될때까지 기다렸다가 보여줌 / model.stram(input, config)->Iterator : 토큰을 생성하면 바로 받아 보여줌

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini")
res = model.stream("AI에 대해서 설명해줘.")
print(type(res))

<class 'generator'>


In [7]:
for token in model.stream("AI에 대해 설명해줘."):
    print(token.content, end="")

AI(인공지능)는 사람의 지능을 모방하거나 대체할 수 있는 컴퓨터 시스템 또는 프로그램을 지칭합니다. AI는 데이터를 분석하고, 학습하며, 문제를 해결하는 등의 기능을 수행할 수 있습니다. AI의 주요 분야는 다음과 같습니다:

1. **기계 학습 (Machine Learning)**: AI의 하위 분야로, 데이터로부터 학습하여 예측하거나 결정을 내리는 알고리즘을 개발합니다. 예를 들어, 스팸 이메일 필터링이나 추천 시스템 등이 기계 학습을 활용합니다.

2. **자연어 처리 (Natural Language Processing, NLP)**: 컴퓨터가 인간의 언어를 이해하고 해석할 수 있도록 하는 기술입니다. 챗봇, 번역기, 음성 인식 시스템 등이 이에 해당합니다.

3. **컴퓨터 비전 (Computer Vision)**: 이미지나 비디오에서 정보를 추출하고 해석하는 기술입니다. 자율주행차, 얼굴 인식 시스템 등이 컴퓨터 비전의 예입니다.

4. **전문 시스템 (Expert Systems)**: 특정 분야에서 전문가의 지식을 적용하여 문제를 해결할 수 있도록 설계된 시스템입니다. 의료 진단 시스템이나 금융 분석 도구 등이 포함될 수 있습니다.

AI는 다양한 산업 분야에서 활용되고 있으며, 의료, 금융, 제조업, 서비스업 등에서 효율성을 높이고 혁신을 촉진하는 데 기여하고 있습니다. 그러나 AI의 발전에는 윤리적 문제나 일자리 대체와 같은 다양한 도전 과제가 동반되기도 합니다.